In [66]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import re
import time
import unidecode

In [67]:
Base = declarative_base()

In [68]:
class Artist(Base):
    __tablename__ = 'artists'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    gender = Column(Text)
    birthday = Column(Integer)
    deathday = Column(Integer)
#     hometown = Column(Text)
    location = Column(Text)
    nationality = Column(Text)
    years = relationship('Year', back_populates='artist')
    genes = relationship('Gene', secondary='artists_genes')

In [69]:
class Year(Base):
    __tablename__ = 'years'
    id = Column(Integer, primary_key=True)
    year = Column(Integer)
    rank = Column(Integer)
    artist_id = Column(Integer, ForeignKey('artists.id'))
    artist = relationship('Artist', back_populates='years')
    totalsold = Column(Integer)
    totallots =  Column(Integer)
    maxprice = Column(Integer)

In [70]:
#if no display_name, name
class Gene(Base):
    __tablename__= 'genes'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    artists = relationship('Artist', secondary='artists_genes')

In [71]:
class ArtistGene(Base):
    __tablename__ = 'artists_genes'
    artist_id = Column(Integer, ForeignKey('artists.id'), primary_key=True)
    gene_id = Column(Integer, ForeignKey('genes.id'), primary_key=True)

In [72]:
engine = create_engine('sqlite:///artsy.db')
Base.metadata.create_all(engine)

In [73]:
Session = sessionmaker(bind=engine)
session = Session()

In [74]:
import requests
import json

client_id = '5a2c7ea90a393fa55c65'
client_secret = '5cda60eedbc43c586621c30d6d19210e'

r = requests.post('https://api.artsy.net/api/tokens/xapp_token',
                      data={'client_id': client_id, 'client_secret': client_secret})

token_text = json.loads(r.text)

token = token_text['token']

headers = {'X-Xapp-Token': token}

list_answer = {}
artists = 'qi-baishi'

# api_url='https://api.artsy.net/api/artists/'+artists.lower()
api_url = 'https://api.artsy.net/api/search?offset=0&q=ZHU+DA'
res = requests.get(api_url, headers=headers)
res.encoding = 'utf-8'

In [75]:
res.json()['_embedded']['results'][0]['_links']['self']['href']

'https://api.artsy.net/api/artists/4ee262199de8920001000530'

In [76]:
def verify_artist(json_thing):
    if json_thing.get('type') != 'other_error':
        return True
    elif json_thing.get('type') == 'other_error':
        return False

# Order is fine, gotta figure 404s
def query_json(artist_name):
    artist_name = [re.sub("[')(.]", '', x)  for x in artist_name]
    artist_name = [unidecode.unidecode(y) for y in artist_name]
    name = "-".join(artist_name)
    api_url='https://api.artsy.net/api/artists/'+name.lower()
    res = requests.get(api_url, headers=headers)
    res.encoding = 'utf-8'
    verifiable = verify_artist(res.json())
    if  verifiable == True:
        #debugging
        print('verified true')
        
#         print(res)
        #########
        return res.json()
    elif verifiable == False :
        new_name = list(reversed(artist_name))
        name = "-".join(new_name)
####added time
        time.sleep(.5)
        api_url='https://api.artsy.net/api/artists/'+name.lower()
        res = requests.get(api_url, headers=headers)
        res.encoding = 'utf-8'
        #debugging
        print('flipped name')
        print(res)
        #########
        if res.status_code == 404:
            search_url = 'https://api.artsy.net/api/search?q='+ "+".join(artist_name)
            res = requests.get(search_url, headers=headers)
            res.encoding = 'utf-8'
            try:
                t = res.json()['_embedded']['results'][0]['type']
#                 [0]['type']
# New Code######################
#                 for result in t[:3]:
#                     if result['type'] == 'artist':
#                         api_url = result['_links']['self']['href']
#                         res = requests.get(api_url, headers=headers)
#                         res.encoding = 'utf-8'
#                         x = session.query(Artist).filter(Artist.name == res.json()['name']).first()
#                         if x == None:
#                             return res.json()
#  New CODE#####################
            except:
                t = []
            if t  != 'artist':
                print('no result found')
                return {'name' : " ".join(artist_name), 'skip':True}  
###added time
            time.sleep(.5)
            api_url = res.json()['_embedded']['results'][0]['_links']['self']['href']
            res = requests.get(api_url, headers=headers)
            res.encoding = 'utf-8'
            #debugging
            print('search query')
            print(res)
            ##########
            return res.json()
        print('No search')
        return res.json()
    

In [77]:
search_url = 'https://api.artsy.net/api/search?q='+ '-'.join(['Xu', 'Cao'])
res = requests.get(search_url, headers=headers)
res.encoding = 'utf-8'

In [78]:
len(res.json()['_embedded']['results'])

10

In [79]:
query_json(['Qi', 'Baishi'])

verified true


{'id': '4edfdf720f64f5000100009a',
 'slug': 'qi-baishi',
 'created_at': '2011-12-07T21:49:38+00:00',
 'updated_at': '2018-11-15T07:03:18+00:00',
 'name': 'Qi Baishi',
 'sortable_name': 'Qi Baishi',
 'gender': 'male',
 'biography': '',
 'birthday': '1864',
 'deathday': '1957',
 'hometown': '',
 'location': '',
 'nationality': 'Chinese',
 'image_versions': ['four_thirds', 'large', 'square', 'tall'],
 '_links': {'thumbnail': {'href': 'https://d32dm0rphc51dk.cloudfront.net/LBtYS1GYJI-a7XikfFWWlA/four_thirds.jpg'},
  'image': {'href': 'https://d32dm0rphc51dk.cloudfront.net/LBtYS1GYJI-a7XikfFWWlA/{image_version}.jpg',
   'templated': True},
  'self': {'href': 'https://api.artsy.net/api/artists/4edfdf720f64f5000100009a'},
  'permalink': {'href': 'http://www.artsy.net/artist/qi-baishi'},
  'artworks': {'href': 'https://api.artsy.net/api/artworks?artist_id=4edfdf720f64f5000100009a'},
  'published_artworks': {'href': 'https://api.artsy.net/api/artworks?artist_id=4edfdf720f64f5000100009a&publishe

In [80]:
def create_genes(artist_json):
    genes_toadd = []
    if artist_json.get('skip'):
        return []
    else:
        x = artist_json['_links']['genes']['href']
    time.sleep(.5)
    genes = requests.get(x, headers=headers)
    gene_json = genes.json()['_embedded']['genes']
    artist_genes = []
    for gene in gene_json:
        if gene['display_name'] != None:
            artist_genes.append(gene['display_name']) 
        else: 
            artist_genes.append(gene['name'])
    gene_objects = []
    for gene in artist_genes:
        existing = session.query(Gene).filter(Gene.name == gene).first()
        if existing:
            gene_objects.append(existing)
        else:
            gene_objects.append(Gene(name=gene))
    return gene_objects

In [81]:
session.query(Artist).filter(Artist.name == None).first()

In [82]:
def create_artist(row):
    name = row['Name']
    artist_json = query_json(name)
    existing = session.query(Artist).filter(Artist.name == artist_json['name']).first()
    if existing:
        tf = session.query(Year).filter(and_(Year.year == row.get('SaleYear'), Year.artist_id == existing.id)).first()
        if tf:
            n = " ".join(row.get('Name'))
            bday = row.get('birthday')
            dday = row.get('deathday')
            gen = []
            return Artist(name=n, birthday=bday,
                         deathday=dday, genes=gen)
        elif existing and not tf:
            return existing
    else:
        n = artist_json.get('name')
        g = artist_json.get('gender')
        bday = artist_json.get('birthday')
        dday = artist_json.get('deathday')
        loc = artist_json.get('location')
        nat = artist_json.get('nationality')
        gen = create_genes(artist_json)
        return Artist(name=n, gender=g, birthday=bday,
                     deathday=dday, location=loc, nationality=nat, genes=gen)

In [83]:
def create_database_year(year):
    for row in year:
        print(row['Rank'])
#         name = row['Name']
        artist_object = create_artist(row)
######added time
        time.sleep(.5)
        year = Year(year=row.get('SaleYear'),
                    rank=row.get('Rank'),
                    artist=artist_object,
                    totalsold=row.get('TotalSold'),
                    totallots=row.get('TotalLots'),
                    maxprice=row.get('MaxPrice'))
        session.add(year)
        session.commit()

In [84]:
from artist07to17 import *
# clean2017top500

In [119]:
create_database_year(clean2007top500)
# up next 2006

1
flipped name
<Response [200]>
No search
2
flipped name
<Response [200]>
No search
3
flipped name
<Response [200]>
No search
4
flipped name
<Response [200]>
No search
5
flipped name
<Response [200]>
No search
6
flipped name
<Response [200]>
No search
7
flipped name
<Response [200]>
No search
8
flipped name
<Response [200]>
No search
9
flipped name
<Response [200]>
No search
10
flipped name
<Response [200]>
No search
11
flipped name
<Response [200]>
No search
12
flipped name
<Response [200]>
No search
13
flipped name
<Response [404]>
search query
<Response [200]>
14
flipped name
<Response [200]>
No search
15
flipped name
<Response [200]>
No search
16
flipped name
<Response [200]>
No search
17
flipped name
<Response [200]>
No search
18
flipped name
<Response [200]>
No search
19
flipped name
<Response [200]>
No search
20
flipped name
<Response [200]>
No search
21
flipped name
<Response [200]>
No search
22
verified true
23
flipped name
<Response [200]>
No search
24
flipped name
<Response 

185
flipped name
<Response [404]>
search query
<Response [200]>
186
flipped name
<Response [200]>
No search
187
flipped name
<Response [404]>
search query
<Response [200]>
188
flipped name
<Response [200]>
No search
189
verified true
190
flipped name
<Response [404]>
search query
<Response [200]>
191
flipped name
<Response [404]>
search query
<Response [200]>
192
flipped name
<Response [200]>
No search
193
flipped name
<Response [200]>
No search
194
flipped name
<Response [200]>
No search
195
flipped name
<Response [200]>
No search
196
flipped name
<Response [200]>
No search
197
flipped name
<Response [200]>
No search
198
flipped name
<Response [404]>
search query
<Response [200]>
199
flipped name
<Response [200]>
No search
200
flipped name
<Response [200]>
No search
201
flipped name
<Response [200]>
No search
202
flipped name
<Response [200]>
No search
203
flipped name
<Response [404]>
search query
<Response [200]>
204
flipped name
<Response [200]>
No search
205
verified true
206
flip

KeyError: 'name'

In [116]:
x = session.query(Year).filter(Year.year == 2008).all()

In [117]:
listy = []
listy1 = []
for y in x:
    listy.append(y.artist_id)
    listy1.append(y.year)

In [118]:
len(set(listy)), len(set(listy1))

(500, 1)

In [27]:
listy[1]

2

In [65]:
g1 = session.query(Gene).filter(Gene.name == 'Contemporary').first().artists
g2 = session.query(Gene).filter(Gene.name == 'Abstract Art').first().artists
artists = []
for artist in g1:
    if artist in g2:
        artists.append(artist)
        
for t in artists:
    print(t.name)
# len(g1), len(g2), len(artists)

Cy Twombly
Gerhard Richter
Yayoi Kusama
Christopher Wool
Mark Grotjahn
Damien Hirst
Agnes Martin
Frank Stella
Pierre Soulages
Lee Ufan
Robert Motherwell
Ellsworth Kelly
Maqbool Fida Husain
Antony Gormley
Zheng Chongbin 郑重宾
V. S. Gaitonde
Wolfgang Tillmans
Kenneth Noland
Enrico Castellani
Günther Förg
KAWS
Richard Serra
Robert Ryman
Mark Bradford
Howard Hodgkin
Atsuko Tanaka
Zhuang Hong Yi
He Xiangyu
Bridget Riley
Richard Diebenkorn
Anish Kapoor
Kazuo Shiraga
Sol LeWitt
Christo
Jesús Rafael Soto
Agostino Bonalumi
Tony Cragg
Wade Guyton
Wojciech Fangor
Franz West
Zhang Yu 張羽
Hsiao Chin 蕭勤
Imi Knoebel
Sadamasa Motonaga
Carlos Cruz-Diez
Sam Gilliam
Laura Owens
Otto Piene
Soojung Park
Dexi Tian
Robert Mangold
Xia Yunfei
Jannis Kounellis
Charles Hossein Zenderoudi
Tauba Auerbach
George Chann
Shen Fan
Luciano Fabro
Jim Dine
Wang Guangle
herman de vries
El Anatsui
Lucien Smith
Oscar Murillo
Pierre Huot


In [74]:
clean2013top500[56]

{'Rank': '57',
 'Name': ['Li', 'Xiongcai'],
 'TotalSold': 29445836,
 'TotalLots': '286',
 'MaxPrice': 12760000,
 'SaleYear': 2013}